## Recommender System - Marketplace Matching

In this notebook, we will: 
- Clean textual data from user-input verbatim posts
- Use a Word2Vec model to calculate document similarities 
- Sort the most similar user input to our training data in order to recommend similar products 
- Save this model in a format that allows us to refresh the testing data

The goal of this project is to create a recommender system to help Pangeans find the right "project" for them, given their profile information. Here, we are using legacy data that is from Pangea V2, when Pangeans were allowed to post services and requests, as well as purchase items on the platform. We are using the User-Inputted Titles to suggest similar services, or in V3, similar "projects". 

In [41]:
#Importing Libraries
import numpy as np
import pandas as pd

import sys
from pandas import DataFrame

import json
from pandas.io.json import json_normalize
import csv

import matplotlib.pyplot as plt
%matplotlib inline 
from matplotlib import colors
from matplotlib.ticker import PercentFormatter

from gensim import corpora
from collections import defaultdict
from pprint import pprint

from gensim.models.word2vec import Word2Vec
from gensim.test.utils import common_texts, get_tmpfile

from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.test.utils import common_texts

import re
from sklearn.feature_extraction.text import CountVectorizer

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
nltk.download('stopwords') ###

import gensim 
from gensim.models import KeyedVectors
import gensim.downloader as api

from operator import itemgetter, attrgetter

from gensim.models.doc2vec import Doc2Vec

import os, sys

from operator import add

from sklearn.preprocessing import normalize

import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from adjustText import adjust_text

from sklearn.manifold import TSNE

import pickle

import json
from pprint import pprint
from pandas.io.json import json_normalize

import operator

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/angelateng/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
def vectorize_and_store_existing_titles():
    
    raw = pd.read_csv("allPostData.csv", header=0);
    
    #we can replace this with a filepath in the future
    titles = raw['title'];
    post_titles = [title for title in titles];
    tokens = [[word for word in title.lower().split()] for title in post_titles];
    
    clean_words = [[word for word in title if word.isalpha()] for title in tokens];
    stoplist = set(stopwords.words('english'));
    
    titles_nostopwords = [[word for word in title if word not in stoplist] for title in clean_words];   
    
    model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary = True); 
    filtered_word_list = [[word for word in title if word in model.vocab] for title in titles_nostopwords];
    
    title_vectors = {}
    for title in filtered_word_list: 
        word_vecs = [model[word] for word in title]
        if len(word_vecs) == 0:
            title_vec = [np.zeros(300)]
        else: 
            title_vec = normalize(sum(word_vecs).reshape(1, -1))
        title_vectors[" ".join(title)] = title_vec[0]
    #do we want to print out the original titles? how to do this?
    vectorized_titles = pd.DataFrame.from_dict(title_vectors)
    
    #can also replace filepath in the future
    vectorized_titles.to_pickle("/Users/angelateng/Dropbox/SharpestMinds/vectorized_titles.pkl")
    return(vectorized_titles)

#vectorize_and_store_existing_titles()


In [3]:
#pd.read_pickle("/Users/angelateng/Dropbox/SharpestMinds/vectorized_titles.pkl")
#sanity check

In [9]:
#can change this directory later

with open('allPostData.json') as fresh_data:
    user_post = json.load(fresh_data)

json_df = pd.DataFrame.from_dict(json_normalize(user_post), orient='columns')

def vectorize_new_title(json_df):
    title = json_df["title"]
    
    json_post_titles = [title for title in title];
    json_tokens = [[word for word in title.lower().split()] for title in json_post_titles];
    
    json_clean_words = [[word for word in title if word.isalpha()] for title in json_tokens];
    
    stoplist = set(stopwords.words('english'));
    json_titles_nostopwords = [[word for word in title if word not in stoplist] for title in json_clean_words];   
    
    model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary = True); 
    json_filtered_word_list = [[word for word in title if word in model.vocab] for title in json_titles_nostopwords];
    
    json_title_vectors = {}
    for title in json_filtered_word_list: 
        json_word_vecs = [model[word] for word in title]
        if len(json_word_vecs) == 0:
            json_title_vec = [np.zeros(300)]
        else: 
            json_title_vec = normalize(sum(json_word_vecs).reshape(1, -1))
        json_title_vectors[" ".join(title)] = json_title_vec[0]
    #do we want to print out the original titles? how to do this?
    json_vectorized_titles = pd.DataFrame.from_dict(json_title_vectors)

    return(json_vectorized_titles)

#vectorize_new_title(json_df)

,teach,long boarding lessons,personal trainer,final cut pro lesson,resume building,motivational pep talk,resume cover letter,bracelet,video production,consulting cases,...,personal advice,recital poster,controller,camp counselor,looking buy printer,econ tutoring,someone vent,work wanted,organize iphone,wholesome memes
0,0.042356,0.001692,0.029323,0.075196,-0.035967,0.058533,-0.038079,-0.054434,0.005258,0.018321,...,0.017751,0.057445,0.075356,0.003362,0.043997,0.045284,0.055331,0.009063,-0.106068,0.061221
1,-0.013683,0.068215,-0.014871,0.013171,0.055128,0.039345,0.083954,-0.118624,-0.009919,0.070316,...,0.000913,0.024768,-0.142169,0.012858,0.018273,0.100476,-0.034940,0.079957,-0.030152,0.010694
2,0.077740,-0.002461,0.005665,0.087968,0.020805,0.016026,0.028446,-0.006933,-0.039008,0.000821,...,-0.019127,0.007129,-0.035237,0.014549,-0.035064,-0.016123,-0.024401,0.055108,-0.041650,0.036840
3,0.028238,0.061446,-0.021717,0.011088,0.010480,0.022407,-0.045808,0.104245,0.027535,0.079201,...,0.030615,0.033786,-0.002069,-0.001021,0.096450,0.118554,0.022339,0.010817,0.014916,0.134803
4,0.042008,0.042063,-0.011790,0.028483,0.029083,-0.016356,0.058750,-0.090894,-0.009688,0.008718,...,-0.040445,0.020293,0.018000,-0.002908,-0.014857,0.030247,-0.085460,0.014106,-0.063472,-0.132645
5,0.044622,0.003252,0.098933,-0.005076,-0.019844,0.000799,0.042181,0.078569,0.090763,0.064804,...,0.101450,-0.000208,-0.079932,0.110623,0.047186,0.090220,0.003723,0.005555,0.024741,-0.000687
6,0.087502,-0.060083,-0.006505,-0.037842,0.025177,0.046602,-0.019311,0.023237,0.011345,0.008339,...,0.049995,0.085959,-0.005148,0.002959,0.024077,-0.016340,0.002978,0.121617,0.001431,0.108510
7,-0.047760,-0.080543,-0.015009,-0.095989,-0.040928,-0.080495,0.023754,-0.110921,-0.082095,-0.101147,...,-0.060113,-0.091995,0.015941,-0.012935,-0.022046,-0.081006,-0.045708,-0.059639,-0.052127,0.072602
8,-0.061704,0.039118,0.049572,0.128256,0.123775,0.024291,0.088941,-0.065731,0.077761,0.047509,...,0.061028,0.012072,-0.006636,-0.041535,0.046959,0.013907,0.057508,-0.039723,0.041446,-0.022860
9,-0.068677,-0.007406,-0.079944,0.047445,-0.067360,0.050344,-0.067454,-0.016433,0.032761,-0.009856,...,-0.056399,0.112581,-0.070780,-0.095826,0.070463,-0.052932,0.034940,0.015778,-0.051207,-0.022786


In [42]:
def rank_existing_titles(json_vectorized_titles):
    #loop over all keys in dict 
    ranked_titles = {}
    other_titles = pd.read_pickle("/Users/angelateng/Dropbox/SharpestMinds/vectorized_titles.pkl")
    #can also use title_vectors.keys() 
    for key in json_vectorized_titles:
        ranked_titles[key] = np.dot(other_titles[key], json_vectorized_titles[key])
    sorted_title_vecs = sorted(ranked_titles.items(), key=operator.itemgetter(1), reverse=True)
    return(sorted_title_vecs)

rank_existing_titles

<function __main__.rank_existing_titles(json_vectorized_titles)>

In [ ]:
vectorize_and_store_existing_titles(); 
vectorize_new_title(json_df);
rank_existing_titles(vectorize_new_title);